In [1]:
import os, sys
os.environ

import pathlib
import textwrap

import google
import google.generativeai as genai
from google.generativeai import types

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import json
import requests 

import minsearch

In [23]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

# Q1. Define function description

In [24]:
get_weather_tool = {
    #"type": "function",
    "name": "get_weather",
    "description": "get usefull weather data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for, e.g., 'Berlin'"
            }
        },
        "required": ["city"],
    }
}

In [25]:
genai.configure()

In [31]:
question = "What's the weather like in Russia?"

developer_prompt = """
You're given a question about weather in some country and your task is to answer it using get weather tool.
If there is no city in the question, but is country, then use data for the capital of the country.

In the answer give the city and the temperature
""".strip()

tools = [get_weather_tool]

model = genai.GenerativeModel('gemini-2.5-flash')
messages = [
        {'role':'model', 'parts': developer_prompt},
        {'role':'user', 'parts': question}
    ]
response = model.generate_content(contents=messages,
                                  tools=types.Tool(function_declarations=tools)) 
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_weather",
                  "args": {
                    "city": "Moscow"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 119,
        "candidates_token_count": 15,
        "total_token_count": 279
      },
      "model_version": "gemini-2.5-flash"
    }),
)

In [42]:
get_weather(response.candidates[0].content.parts[0].function_call.args['city'])

26.6

# Q2. Adding another tool

In [44]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [45]:
add_weather_to_data_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "add new weather data to database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for, e.g., 'Berlin'"
            },
            "weather": {
                "type": "float",
                "description": "Weather data point"
            }
        },
        "required": ["city", "weather"],
    }
}

# Q3. Install FastMCP

In [2]:
import fastmcp
from fastmcp import FastMCP
print(fastmcp.__version__)

2.10.5


# Q4. Simple MCP Server

In [7]:
'''# weather_server.py
import fastmcp
from fastmcp import FastMCP
import random
import asyncio

mcp = FastMCP("Weather 🚀")

known_weather_data = {
    'berlin': 20.0
}

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()
    '''

╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Weather 🚀                                          │
│    📦 Transport:       Streamable-HTTP                                     │
│    🔗 Server URL:      http://127.0.0.1:8000/mcp/                          │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

[07/16/25 11:15:41] INFO     Starting MCP server 'Weather 🚀' with transport 'http' on               ]8;id=818138;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=753534;file:///usr/local/python/3.12.1/lib/python3.12/site-packages/fastmcp/server/server.py#1448\1448]8;;\
                             http://127.0.0.1:8000/mcp/                                                            

/usr/local/python/3.12.1/lib/python3.12/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/usr/local/python/3.12.1/lib/python3.12/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [66316]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
ERROR:    Cancel 0 running task(s), timeout graceful shutdown exceeded
INFO:     Finished server process [66316]


# Q6. Client

In [10]:
from fastmcp import Client
import weather_server  # Import your server module

async def main():
    # Connect to the server (using the existing mcp instance)
    async with Client(weather_server.mcp) as mcp_client:
        # Get the list of available tools
        tools = await mcp_client.list_tools()
        print("Available Tools:", tools)

# Run in Jupyter
await main()

Available Tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem